# INSTALLING PACKAGES

We install all the necessary packages to build the model

In [ ]:
!pip install glom

In [ ]:
!pip install yfinance

In [ ]:
!pip install newspaper3k

In [ ]:
!pip install bs4

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

In [ ]:
!pip install tweepy

In [ ]:
!pip install htmldate

In [ ]:
!pip install sentencepiece

In [ ]:
!pip install torch==1.9.0+cu111 torchvision==0.10.0+cu111 torchaudio===0.9.0 -f https://download.pytorch.org/whl/torch_stable.html

In [ ]:
!pip install transformers[torch]==4.3


# IMPORTING PACKAGES

Import packages for performing data extraction, cleaning, visulaization, and modelling

In [ ]:
import tweepy, json

In [ ]:
from glom import glom

In [ ]:
import numpy as np

In [ ]:
import pandas as pd

In [ ]:
import math

In [ ]:
from datetime import datetime, timedelta

In [ ]:
import yfinance as yf

In [ ]:
from bs4 import BeautifulSoup

In [ ]:
import requests

In [ ]:
import re

In [ ]:
from newspaper import *

In [ ]:
from htmldate import find_date

In [ ]:
from nltk.corpus import stopwords

In [ ]:
from transformers import PegasusTokenizer, PegasusForConditionalGeneration

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [ ]:
from transformers import pipeline

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

# COLLECTING TWITTER DATA

Getting developer access from twitter to extract historical tweets for upto 7 days

NOTE: The keys below are private and are to be kept confidential and not be misused without the owner's authorization

In [ ]:
api_key = "D3a1PVON8wECC8ZB781LXEY0I"
api_secret_key = "lrrXjpTn0P2Rj1R1lH3cVn1KUA4xrTZyrRzb9Jfgwx1bKPC5Y2"
access_token = "1319681101-suDjXaeuUGiCMVoQ3ZHmwIv84CNkXbkvuSE39JK"
access_token_secret = "IvLRZh7i2PaxmXr4riVb7QZoLBOzqS3MJfDEEq9nX9oYU"

In [ ]:
auth = tweepy.OAuthHandler(api_key,api_secret_key)
auth.set_access_token(access_token,access_token_secret)
api = tweepy.API(auth)

In [ ]:
def get_tweets(ticker_list, count):
  tweets = pd.DataFrame()

  for i in range(len(ticker_list)):
    results = api.search(q=ticker_list[i], lang='en', count=count)
    results = pd.DataFrame(results)
    temp = pd.DataFrame()

    temp['Timestamp'] = results[0].apply(lambda row: glom(row, 'created_at'))
    temp['Title'] = results[0].apply(lambda row: glom(row, 'text'))
    temp['Ticker'] = ticker_list[i]
    temp['Tweet ID'] = results[0].apply(lambda row: glom(row, 'id'))
    temp['User ID'] = results[0].apply(lambda row: glom(row, 'user.id'))
    temp['Name'] = results[0].apply(lambda row: glom(row, "user.screen_name"))
    temp['Followers'] = results[0].apply(lambda row: glom(row, 'user.followers_count'))
    temp['Location'] = results[0].apply(lambda row: glom(row, 'user.location'))
    tweets = tweets.append(temp)
    tweets.drop_duplicates(inplace=True)
  return tweets

Ticker list includes the name of all stocks under NIFTY Bank

Tweets containing any of these keywords are extracted

In [ ]:
ticker_list = ["NIFTY BANK", 'SBIN', 'IDBI', 'AXISBANK', 'HDFCBANK', 'KOTAKBANK', 'PNB', 'ICICIBANK', 'BARODA', 'CANARA', 'INDUSIND', 'BANKINDIA', 'UNION']
count = 1000

tweets = get_tweets(ticker_list=ticker_list, count=count)

In [ ]:
tweets.head()

In [ ]:
def firstandlastdate(dataframe):
  first = np.min(dataframe['Timestamp'])
  last = np.max(dataframe['Timestamp'])
  return first, last

In [ ]:
first_tw, last_tw = firstandlastdate(tweets)

print('Starting: '+str(first_tw) +'\n' +'Ending: '+str(last_tw))

In [ ]:
tweets.info()

# EXTRACTING NEWS

Functions are created to crawl webpages to get URLs, filter them, and feed it through the data extraction algorithm to get news data

In [ ]:
def search_for_stocks_news_urls(ticker, url_list):
    for url in url_list:
        search_url = url.format(ticker)
        r = requests.get(search_url)
        soup = BeautifulSoup(r.text, 'html.parser')
        atags = soup.find_all('a')
        hrefs = [tag['href'] if tag.has_attr('href') else '' for tag in atags]
    return hrefs

In [ ]:
def strip_unwanted_urls(urls, exclude_list):
    val = []
    for url in urls: 
        if 'https://' in url and not any(exclude_word in url for exclude_word in exclude_list):
            res = re.findall(r'(https?://\S+)', url)[0].split('&')[0]
            val.append(res)
    return list(set(val))

Newspaper3k package is used exclusively to find the publish date of each article extracted

In [ ]:
def publish_date(URLs):
    publishdate = []
    for url in URLs:
        try:
            publishdate.append(str(find_date(url)))
        except:
            publishdate.append("None")
            pass
    return publishdate

In [ ]:
def get_news_data(ticker_list, cleaned_urls):
    articles_info = []
    
    for ticker in ticker_list:
        links = cleaned_urls[ticker]
    
        for i in links:
            article_dict = {}
            article = Article(i)
            article.download()

            article_dict["link"] = i
            article_dict["Ticker"] = ticker

            try:
                article.parse()
                article_dict["Text"] = article.text
                article_dict["Title"] = article.title
                article.nlp()

            except ArticleException:
                article_dict["Text"] = np.nan
                article_dict["Title"] = np.nan

            articles_info.append(article_dict)

    news_data = pd.DataFrame(articles_info)
    return news_data

In [ ]:
ticker_list = ["NIFTY BANK", 'SBIN', 'IDBI', 'AXISBANK', 'HDFCBANK', 'KOTAKBANK', 'PNB', 'ICICIBANK', 'BARODA', 'CANARA', 'INDUSIND', 'BANKINDIA', 'UNION']

url_list = ["https://www.google.com/search?q={}&tbm=nws"] 

raw_urls = {ticker:search_for_stocks_news_urls(ticker=ticker, url_list=url_list) for ticker in ticker_list}

In [ ]:
links = []
for ticker in ticker_list:
    print('Number of URLs in '+ticker + ': '+str(len(raw_urls[ticker])))
    for link in raw_urls[ticker]:
        links.append(link)
print('Total number of raw URLs: '+str(len(links)))

In [ ]:
exclude_list = ['maps', 'policies', 'preferences', 'accounts', 'support']

cleaned_urls = {ticker:strip_unwanted_urls(raw_urls[ticker], exclude_list) for ticker in ticker_list}

In [ ]:
links = []
for ticker in ticker_list:
    print('Number of URLs in '+ticker + ': '+str(len(cleaned_urls[ticker])))
    for link in cleaned_urls[ticker]:
        links.append(link)
print('Total number of cleaned URLs: '+str(len(links)))

In [ ]:
articles_publishdate = {ticker:publish_date(cleaned_urls[ticker]) for ticker in ticker_list}
articles_publishdate

In [ ]:
dates = []
for ticker in ticker_list:
    count = 0
    for value in articles_publishdate[ticker]:
        if value != "None":
            count = count + 1
    print('Number of URLs in '+ticker + ' with dateofpub: '+str(count))
    for date in articles_publishdate[ticker]:
        dates.append(link)  

In [ ]:
dates = []
for ticker in ticker_list:
    print('Number of URLs in '+ticker + ' with dateofpub: '+str(len(articles_publishdate[ticker])))
    for date in articles_publishdate[ticker]:
        dates.append(link)

In [ ]:
news_data = get_news_data(ticker_list=ticker_list, cleaned_urls=cleaned_urls)

In [ ]:
dates = []
for ticker in ticker_list:
    for l in articles_publishdate[ticker]:
        dates.append(l)

news_data['Timestamp'] = dates

Articles without publish date are removed and are converted to datetime datatype so that it uniform with the stock data

In [ ]:
news_data = news_data[news_data['Timestamp'] != 'None']

In [ ]:
news_data['Timestamp'] = news_data['Timestamp'].apply(lambda date: datetime(int(date[:4]), int(date[5:7]), int(date[8:])))

In [ ]:
def timestamp_adjuster(timestamp, start, delta):

    if timestamp >= datetime(timestamp.year,timestamp.month,timestamp.day,9,15) and timestamp <= datetime(timestamp.year,timestamp.month,timestamp.day,15,15):
        timestamp = start + math.ceil((timestamp - start) / delta) * delta

    elif timestamp > datetime(timestamp.year,timestamp.month,timestamp.day,15,15) and timestamp < datetime(timestamp.year,timestamp.month,timestamp.day)+timedelta(days=1):
        timestamp = datetime(timestamp.year,timestamp.month,timestamp.day,9,15) + timedelta(days=1)
  
    else:
        timestamp = datetime(timestamp.year,timestamp.month,timestamp.day,9,15)
  
    return timestamp

In [ ]:
news_data['Timestamp'] = news_data['Timestamp'].apply(lambda x: datetime.strptime(x.strftime('%Y-%m-%d %H:%M:%S'), '%Y-%m-%d %H:%M:%S'))

news_data['Timestamp'] = news_data['Timestamp'].apply(lambda x: timestamp_adjuster(x, start=datetime(2020,1,1,0,15), delta=timedelta(hours=1)))

In [ ]:
news_data.dropna(subset=['Text'], axis=0, inplace=True)

In [ ]:
news_data['Text'] = news_data['Text'].apply(lambda x: str(x))

In [ ]:
news_data.index = range(0, len(news_data))

In [ ]:
news_data.head()

In [ ]:
first_ne, last_ne = firstandlastdate(news_data)

print('Starting: '+str(first_ne) +'\n' +'Ending: '+str(last_ne))



# GETTING STOCKS DATA

Stock data with a resolution of 1 hour is extracted from Yahoo Finance using appropriate tickers for the period where news data and tweets are available

In [ ]:
def get_stocksdata(ticker_list, start, end, period, resolution):
    stocks_data = pd.DataFrame()
    for i in range(len(ticker_list)):
        temp = yf.download(tickers=[ticker_list[i-1]], start=start, end=end, auto_adjust=True, period=period, interval=resolution)
        temp['Timestamp'] = temp.index
        temp['Symbol'] = ticker_list[i-1]
        stocks_data = stocks_data.append(temp)
    return stocks_data

In [ ]:
stocks_ticker_list = ["^NSEBANK", 'SBIN.NS', 'IDBI.NS', 'AXISBANK.NS', 'HDFCBANK.NS', 'KOTAKBANK.NS', 'PNB.NS', 'ICICIBANK.NS', 'BANKBARODA.NS', 'CANBK.NS', 'INDUSINDBK.NS', 'BANKINDIA.NS', 'UNIONBANK.NS'] 
start = min(first_tw, first_ne)
end = min(last_tw, last_ne)
period = '1d'
resolution = '1h'

stocks_data = get_stocksdata(ticker_list=stocks_ticker_list, start=start, end=end, period=period, resolution=resolution)

In [ ]:
stocks_data.index = range(1, len(stocks_data)+1)

In [ ]:
stocks_data['Date'] = stocks_data['Timestamp'].apply(lambda x: x.date)

stocks_data['Time'] = stocks_data['Timestamp'].apply(lambda x: x.time)

stocks_data['Timestamp'] = stocks_data['Timestamp'].apply(lambda x: datetime.strptime(x.strftime('%Y-%m-%d %H:%M:%S'), '%Y-%m-%d %H:%M:%S'))

In [ ]:
stocks_symbol_map = {'NIFTY BANK':"^NSEBANK", 'SBIN':'SBIN.NS', 'IDBI':'IDBI.NS', 'AXISBANK':'AXISBANK.NS', 'HDFCBANK':'HDFCBANK.NS', 
                     'KOTAKBANK':'KOTAKBANK.NS', 'PNB':'PNB.NS', 'ICICIBANK':'ICICIBANK.NS', 'BARODA':'BANKBARODA.NS', 
                     'CANARA':'CANBK.NS', 'INDUSIND':'INDUSINDBK.NS', 'BANKINDIA':'BANKINDIA.NS', 'UNION':'UNIONBANK.NS'}
symbol_stocks_map = {k:v for v,k in stocks_symbol_map.items()}

stocks_data['Ticker'] = stocks_data['Symbol'].map(symbol_stocks_map)

In [ ]:
stocks_data.head()

In [ ]:
stocks_data = stocks_data[['Open', 'Volume', 'Ticker', 'Timestamp']]

In [ ]:
for i in range(1, len(stocks_data['Timestamp'])+1):
  temp = stocks_data['Timestamp'][i]
  stocks_data['Timestamp'][i] = datetime(temp.year, temp.month, temp.day, temp.hour, temp.minute, temp.second)

In [ ]:
stocks_data[stocks_data['Ticker'] == 'NIFTY BANK']



# SUMMARIZING TEXT AND PERFORMING SENTIMENT ANALYSIS

In [ ]:
tweets['Timestamp'] = tweets['Timestamp'].apply(lambda x: timestamp_adjuster(timestamp=x, start=datetime(2020,1,1,0,15), delta=timedelta(hours=1)))

In [ ]:
tweets = tweets[['Ticker', 'Title', 'Timestamp']]

In [ ]:
tweets.columns = ['Ticker', 'Text Summary', 'Timestamp']

News data is summarized using a pre-trained model

In [ ]:
summarizer = pipeline("summarization")

In [ ]:
Text_Summary = []
for i in range(0, len(news_data['Text'])):
  summary = summarizer(news_data['Text'][i][:512], max_length=50, min_length=25, do_sample=False)[0]['summary_text']
  Text_Summary.append(summary)

In [ ]:
news_data['Text Summary'] = Text_Summary

In [ ]:
news_and_tweets = news_data[['Ticker', 'Text Summary', 'Timestamp']].append(tweets)

In [ ]:
news_and_tweets.index = range(0, len(news_and_tweets['Text Summary']))

In [ ]:
news_and_tweets.head()

Irrelavant and insignificant words are removed and fed into FINBERT model for performing Sentiment Analysis

FINBERT is a pre-trained NLP model with weights for a corpus of words from the English language with more relevance to financial context

In [ ]:
stop = stopwords.words('english')
stop[:5]

In [ ]:
news_and_tweets['Text Summary'] = news_and_tweets['Text Summary'].apply(lambda x: ' '.join([w for w in x.split() if w not in stop]))

In [ ]:
news_and_tweets.head()

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")

tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")

classifier = pipeline('sentiment-analysis', model=model, tokenizer=tokenizer)

In [ ]:
news_and_tweets = news_and_tweets.assign(sentiment = lambda x: x['Text Summary'].apply(lambda s: classifier(s))).assign(label = lambda x: x['sentiment'].apply(lambda s: (s[0]['label'])),score = lambda x: x['sentiment'].apply(lambda s: (s[0]['score'])))

In [ ]:
news_and_tweets.head()

In [ ]:
for i in range(0, len(news_and_tweets['Timestamp'])):
  temp = news_and_tweets['Timestamp'][i]
  news_and_tweets['Timestamp'][i] = datetime(temp.year, temp.month, temp.day, temp.hour, temp.minute, temp.second)

In [ ]:
news_and_tweets.drop('sentiment', axis=1, inplace=True)

In [ ]:
news_and_tweets.columns = ['Ticker', 'Text Summary', 'Timestamp', 'Sentiment', 'Sentiment Score']

Sentiment is represented in trading terms based on the polarity scores

Articles with neutral sentiments or low polarity scores are treated as neutral

In [ ]:
for i in range(0, len(news_and_tweets['Sentiment'])):
  if news_and_tweets['Sentiment'][i] == 'positive' and news_and_tweets['Sentiment Score'][i] >= 0.6:
    news_and_tweets['Sentiment'][i] = 'Buy'
  elif news_and_tweets['Sentiment'][i] == 'negative' and news_and_tweets['Sentiment Score'][i] >= 0.6:
    news_and_tweets['Sentiment'][i] = 'Sell'
  else:
    news_and_tweets['Sentiment'][i] = 'None'

Maximum budget per transaction is limited to Rs.10000 and the cost of transaction regresses with lower sentiment polarity scores

All these figures are arbitary

In [ ]:
budget_per_share = 10000
news_and_tweets['Quantity'] = np.zeros(len(news_and_tweets['Ticker']))

for i in range(0, len(news_and_tweets['Sentiment Score'])):
  try:
    tstamp = news_and_tweets.loc[i]['Timestamp']
    tick = news_and_tweets.loc[i]['Ticker']
    if news_and_tweets['Sentiment Score'][i] >= 0.9:
      news_and_tweets['Quantity'][i] = round((budget_per_share) / (stocks_data[(stocks_data['Ticker'] == tick) & (stocks_data['Timestamp'] == tstamp)]['Open']))
    elif news_and_tweets['Sentiment Score'][i] >= 0.8 and news_and_tweets['Sentiment Score'][i] < 0.9:
      news_and_tweets['Quantity'][i] = round((budget_per_share-2000) / (stocks_data[( stocks_data['Ticker'] == tick) & (stocks_data['Timestamp'] == tstamp)]['Open']))
    elif news_and_tweets['Sentiment Score'][i] >= 0.7 and news_and_tweets['Sentiment Score'][i] < 0.8:
      news_and_tweets['Quantity'][i] = round((budget_per_share-4000) / (stocks_data[(stocks_data['Ticker'] == tick) & (stocks_data['Timestamp'] == tstamp)]['Open']))
    elif news_and_tweets['Sentiment Score'][i] >= 0.6 and news_and_tweets['Sentiment Score'][i] < 0.7:
      news_and_tweets['Quantity'][i] = round((budget_per_share-6000) / (stocks_data[(stocks_data['Ticker'] == tick) & (stocks_data['Timestamp'] == tstamp)]['Open']))
    else:
      news_and_tweets['Quantity'][i] = 0
  except:
    news_and_tweets['Quantity'][i] = 0

In [ ]:
news_and_tweets.head()

# MAPPING DATA AND AUTOMATION

In [ ]:
nt = news_and_tweets
nt['Timestamp'] = nt['Timestamp'].apply(lambda x: pd.to_datetime(x))

In [ ]:
st = stocks_data
st['Timestamp'] = st['Timestamp'].apply(lambda x: pd.to_datetime(x))

News data and stock data are merged on timestamp and by ticker values

In [ ]:
df = pd.merge_asof(st.sort_values('Timestamp'), nt.sort_values('Timestamp'), by='Ticker', on='Timestamp', direction='forward')

In [ ]:
df.head()

In [ ]:
temp = df[['Timestamp', 'Open', 'Quantity', 'Ticker', 'Sentiment']]
for i in range(1, len(temp['Timestamp'])):
  if temp['Ticker'][i] != 'NIFTY BANK':
    temp['Open'][i] = temp['Open'][i-1]

fig, ax = plt.subplots(figsize=(25,5))
plt.plot(temp['Timestamp'], temp['Open'], marker='o', markersize=1)
ax.scatter(temp[(temp['Sentiment']=='Buy')]['Timestamp'], temp[(temp['Sentiment']=='Buy')]['Open'], marker='^', c='g')
ax.scatter(temp[(temp['Sentiment']=='Sell')]['Timestamp'], temp[(temp['Sentiment']=='Sell')]['Open'], marker='v', c='r')
ax.set_ylim([min(temp[temp['Ticker']=='NIFTY BANK']['Open']), max(temp[temp['Ticker']=='NIFTY BANK']['Open'])])
ax.set_xlabel('Timestamp')
ax.set_ylabel('Rupees')
ax.set_title('NIFTY BANK')

In [ ]:
fig, axes = plt.subplots(nrows=12, ncols=1, figsize=(25,50))
tl = ticker_list

for t in range(0, len(tl)-1):
  temp = df[['Timestamp', 'Open', 'Quantity', 'Ticker', 'Sentiment']]
  for i in range(1, len(temp['Timestamp'])):
    if temp['Ticker'][i] != tl[t]:
      temp['Open'][i] = temp['Open'][i-1]

  axes[t].plot(temp['Timestamp'], temp['Open'], c='b')
  axes[t].scatter(temp[(temp['Sentiment']=='Buy') & (temp['Ticker']==tl[t])]['Timestamp'], temp[(temp['Sentiment']=='Buy') & (temp['Ticker']==tl[t])]['Open'], marker='^', c='g')
  axes[t].scatter(temp[(temp['Sentiment']=='Sell') & (temp['Ticker']==tl[t])]['Timestamp'], temp[(temp['Sentiment']=='Sell') & (temp['Ticker']==tl[t])]['Open'], marker='v', c='r')
  axes[t].set_ylim([min(temp[temp['Ticker']==tl[t]]['Open']), max(temp[temp['Ticker']==tl[t]]['Open'])])
  axes[t].set_xlabel('Timestamp')
  axes[t].set_ylabel('Rupees')
  axes[t].set_title(tl[t])
  plt.tight_layout()

Now, we simulate the trading through the period to arrive at the amount invested and earned based on investor sentiments

In [ ]:
portfolio = {ticker:0 for ticker in ticker_list}
bank = {ticker:0 for ticker in ticker_list}
temp = df.dropna()

for i in range(len(temp['Timestamp'])-1):
  n_buy = {ticker:0 for ticker in ticker_list}
  n_sell = {ticker:0 for ticker in ticker_list}
  if temp['Sentiment'][i] == 'Buy' and n_buy[temp['Ticker'][i]] <= 3:
    buy_qty = temp['Quantity'][i] 
    cost_buy = buy_qty * temp['Open'][i]
    if portfolio[temp['Ticker'][i]] * temp['Open'][i] <= 10000:
      portfolio[temp['Ticker'][i]] += buy_qty
      bank[temp['Ticker'][i]] -= cost_buy
      n_buy[temp['Ticker'][i]] += 1
      n_sell[temp['Ticker'][i]] -= 1
  elif temp['Sentiment'][i] == 'Sell' and n_sell[temp['Ticker'][i]] <= 3:
    sell_qty = temp['Quantity'][i] 
    cost_sell = sell_qty * temp['Open'][i]
    if portfolio[temp['Ticker'][i]] * temp['Open'][i] <= 10000:
      portfolio[temp['Ticker'][i]] -= sell_qty
      bank[temp['Ticker'][i]] += cost_sell
      n_buy[temp['Ticker'][i]] -= 1
      n_sell[temp['Ticker'][i]] += 1 
  else:
    bank[temp['Ticker'][i]] += 0

sum = 0
value = 0
for ticker in {ticker:0 for ticker in ticker_list[1:]}:
  quantity = portfolio[ticker]
  price = round(temp[temp['Ticker'] == ticker].iloc[-1]['Open'], 2)
  value = round(quantity * price, 2)
  print(ticker +' : \n' +'Quantity = ' +str(quantity) +'\n' +'Price = ' +str(price) +'\n' +'Value = ' +str(value) +'\n' +'Bank = ' +str(round(bank[ticker])))
  print('\n')
  sum += value

bank_total = 0
for i in ([bank[ticker] for ticker in ticker_list[1:]]):
  bank_total += i

print('\n')  
print('Total Value (Position) of Portfolio at end: \n' 
      +'Bank = ' +str(round(bank_total)) +'\n'
      +'Portfolio = ' +str(round(sum)) +'\n'
      +'Total = ' +str(round(bank_total + sum)))

Similarly, we automate an alternate scenario where the same amount of money is invested in the NIFTY Bank index fund at the same time instead of individual stocks

In [ ]:
bank = 0
temp = df.dropna()
index_qty = 0

for i in range(len(temp['Timestamp'])-1):
  if temp['Sentiment'][i] == 'Buy':
    buy_qty = temp['Quantity'][i] 
    cost_buy = buy_qty * temp['Open'][i]
    index_price = temp[(temp['Ticker'] == 'NIFTY BANK') & (temp['Timestamp'] == temp['Timestamp'][i])].iloc[-1]['Open']
    if cost_buy <= 10000:
      index_qty += cost_buy / index_price
      bank -= cost_buy
  elif temp['Sentiment'][i] == 'Sell':
    sell_qty = temp['Quantity'][i]
    cost_sell = sell_qty * temp['Open'][i]
    index_price = temp[(temp['Ticker'] == 'NIFTY BANK') & (temp['Timestamp'] == temp['Timestamp'][i])].iloc[-1]['Open']
    index_qty -= cost_sell / index_price
    bank += cost_sell
  else:
    bank += 0

price = temp[temp['Ticker'] == 'NIFTY BANK'].iloc[-1]['Open']
index_fund = index_qty * price
print('Bank = ' +str(round(bank)) +'\n')
print('Index Quantity = ' +str(index_qty) +' Index Price = ' +str(round(price)) +' Index Fund = ' +str(round(index_fund)) +'\n')
print('Total Value (Position) of NIFTY BANK fund at end = ' +str(round(bank + index_fund)))

It is evident that the results are better when traded using the model